# first complete the remaining part of this notebook(create a proper rag)



# second assisgnment is: take a multiple pdf with text,image,table
1. fetch the data from pdf
2. at lesat there should be 200 pages
3. if chunking(use the sementic chunking technique) required do chunking and then embedding
4. store it inside the vector database(use any of them 1. mongodb 2. astradb 3. opensearch 4.milvus) ## i have not discuss then you need to explore
5. create a index with all three index machnism(Flat, HNSW, IVF) ## i have not discuss then you need to explore
6. create a retriever pipeline
7. check the retriever time(which one is fastet)
8. print the accuray score of every similarity search
9. perform the reranking either using BM25 or MMR ## i have not discuss then you need to explore
10. then write a prompt template
11. generte a oputput through llm
12. render that output over the DOCx ## i have not discuss then you need to explore
as a additional tip: you can follow rag playlist from my youtube

after completing it keep it on your github and share that link on my  mail id:
snshrivas3365@gmail.com

and share the assignment in your community chat as well by tagging krish and sunny

deadline is: till firday 9PM
   

# # RAG Assignment Part a
### 1 Load the PDF

In [1]:
# Load the PDF
from langchain_community.document_loaders import PyPDFLoader
FILE_PATH=r"/Users/ankita/Documents/Krish Naik Academy/Agentic Batch 2/RAG Assignment/ISLR.pdf"
loader=PyPDFLoader(FILE_PATH)

✅ Use loader.load() if you're dealing with a small PDF and don't need to worry about memory.

✅ Use loader.alazy_load() if you’re dealing with a large PDF (e.g., 600 pages) and want to process it incrementally without blocking.

In [2]:
# Length of PDF
#pages=loader.load() #Synchronous method
#len(pages)

pages = []
async for page in loader.alazy_load():
    pages.append(page)

pages = pages[:200]
# my page size is 613, for coding i used 200 only

len(pages)

200

### 2 Chunking

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

split_docs = splitter.split_documents(pages)
len(split_docs)


611

### 3 Store in Vector DB

In [4]:
# Embed with OpenAI
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=768)

#from langchain_huggingface import HuggingFaceEmbeddings
#embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text="This is a tutorial on OPENAI embeddings"
query_result=embeddings.embed_query(text)
len(query_result)

768

Pinecone Vector Database:

In [5]:
from pinecone import Pinecone

import os
pinecone_api_key=os.getenv("PINECONE_API_KEY")
pc=Pinecone(api_key=pinecone_api_key)

index_name="agenticbatch2"
print(pc.has_index(index_name))

#creating a index
if not pc.has_index(index_name):
    pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")    

)
    
#loading the index
index=pc.Index(index_name)

True


/opt/miniconda3/envs/milvus_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_pinecone import PineconeVectorStore
vector_store=PineconeVectorStore(index=index,embedding=embeddings)
vector_store

results = vector_store.similarity_search("what is a machine-learning?")
results

[Document(id='4a81621a-cab4-4b9d-8ee0-40ca4781ce29', metadata={'creationdate': '2023-08-14T19:08:47+00:00', 'creator': 'Adobe InDesign 17.4 (Windows)', 'moddate': '2024-01-20T13:15:36-08:00', 'page': 15.0, 'page_label': '16', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'source': '/Users/ankita/Documents/Krish Naik Academy/Agentic Batch 2/RAG Assignment/ISLR.pdf', 'total_pages': 613.0}, page_content='generalized additive models. Neural networksgained popularity\nin the 1980s, andsupport vector machinesarose in the 1990s.\nSince that time, statistical learning has emerged as a new subfield in\nstatistics,focusedonsupervisedandunsupervisedmodelingandprediction.\nIn recent years, progress in statistical learning has been marked by the\nincreasingavailabilityofpowerfulandrelativelyuser-friendlysoftware,such\nas the popular and freely availablePython system. This has the potential to\ncontinue the transformation of the field from a set of techniques used and'),
 Documen

In [7]:
# Add book pages to vector store
vector_store.add_documents(documents=split_docs)

['30b126a8-58ff-46f2-aedd-246e3e1ffdaa',
 '318eaf40-5fe5-4564-892e-807fc218d1d8',
 'e98e7921-6a20-4d64-8e96-d95d9d97e2ac',
 'd9a345ad-0667-4434-9437-a476bacf6dd5',
 '78ac4536-90c5-4113-95c8-d93bd0caa342',
 'a8887382-93cb-4df4-addc-493451196121',
 '3d62960a-dfed-4f00-884e-5dd02c37bb79',
 '70398398-5502-4ca2-b712-be1dbbdccef2',
 '34c3461c-2158-42df-9c69-ad102859bfd5',
 '890655dd-cb74-4b57-a07a-2cf29d6f5fd8',
 'd5b9ff85-af99-4f44-8bdf-5058458471f4',
 '6f924442-9c7e-4e57-a9ef-66517c1d032e',
 'c94d7a9d-cb73-452b-87fd-2ec4330f73d6',
 '654fafea-b441-4922-8a83-0b42917d702c',
 '108880f3-0339-4d7d-8112-207186afddf1',
 'e8d61d8f-4e09-4e28-bb76-fc49bd026dbd',
 'cda8b5bd-42ad-49e8-8319-7cf2f80c8abc',
 '5055a481-ea94-4b60-928a-a7a9b4074680',
 '0f7f358c-9f8e-4860-832d-714273c0b871',
 '6d61f004-f693-4427-94a7-1007c9bca03b',
 'ef590da0-9bee-4eee-85d2-fc2743bc0598',
 '996639f8-bafd-41f9-8dee-1119b66b4ab8',
 '83255199-4121-41ed-8077-4692b2e4f4a1',
 '1bef14e1-dc77-4cbc-b903-469730948e53',
 'ddf036c2-cd87-

In [8]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [9]:
# Test of pipeline
retriever.invoke("what is a machine-learning")

[Document(id='4a81621a-cab4-4b9d-8ee0-40ca4781ce29', metadata={'creationdate': '2023-08-14T19:08:47+00:00', 'creator': 'Adobe InDesign 17.4 (Windows)', 'moddate': '2024-01-20T13:15:36-08:00', 'page': 15.0, 'page_label': '16', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'source': '/Users/ankita/Documents/Krish Naik Academy/Agentic Batch 2/RAG Assignment/ISLR.pdf', 'total_pages': 613.0}, page_content='generalized additive models. Neural networksgained popularity\nin the 1980s, andsupport vector machinesarose in the 1990s.\nSince that time, statistical learning has emerged as a new subfield in\nstatistics,focusedonsupervisedandunsupervisedmodelingandprediction.\nIn recent years, progress in statistical learning has been marked by the\nincreasingavailabilityofpowerfulandrelativelyuser-friendlysoftware,such\nas the popular and freely availablePython system. This has the potential to\ncontinue the transformation of the field from a set of techniques used and'),
 Documen

### 4 Load LLM

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [11]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [12]:
# Output
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [15]:
rag_chain.invoke("what is Artificial Neural Network? Explain CNN in detail")


'I am sorry, but this document only provides a table of contents and does not contain an explanation of artificial neural networks or CNNs.  Therefore, I cannot answer your question.'